### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from collections import OrderedDict

### Function to estimate y hat given coefficients beta and data X

In [2]:
def estimate_yhat(beta, X):
    """ (array, array) -> array
    Returns the estimate y hat computed using the coefficients beta and input data array X   
    """
    yhat = np.matmul(beta, X.T)
    return yhat.reshape(yhat.shape[0],1)

### Function to estimate RSS given coefficients beta, data containing independent vars X and dependent var y

In [3]:
def RSS(beta, X, y):
    """ (array, array, array) -> float
    Returns the Sum of Squared errors calculated using the coefficients beta, input data array X and the corresponding actual value of y    
    """
    yhat = estimate_yhat(beta, X)    
    return np.sum((y - yhat) ** 2)

### Reading the input wine dataset

In [4]:
winedata = pd.read_csv('winequality-red.csv')

### Splitting independent and dependent variables

In [5]:
X = winedata[winedata.columns[:-1]]
y = winedata[['quality']]

### Sampling 80% of data randomly for training and keeping the remainder for test

In [6]:
X_train = X.sample(frac=0.8)
y_train = y.iloc[X_train.index]
X_test = X.iloc[list(set(X.index) - set(X_train.index))]
y_test = y.iloc[X_test.index]

In [7]:
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

### Initialize beta0 to random values from the standard normal distribution

In [8]:
beta0 = np.random.normal(0, 1, X_train.shape[1])

### Finding the optimal values of beta using the minimize function

In [9]:
res = minimize(fun=RSS, x0=beta0, args=(X_train,y_train))
beta_hat = res.x

### Predicting y for both training and test using the optimized values beta hat

In [10]:
y_train_pred = estimate_yhat(beta_hat, X_train)
y_test_pred = estimate_yhat(beta_hat, X_test)

### Computing RSS for both training and test using the optimized values beta hat

In [11]:
RSS_train = RSS(beta_hat, X_train, y_train)
RSS_test = RSS(beta_hat, X_test, y_test)

### Printing RSS values and beta hat

In [12]:
print ("The RSS value for the training data set is {0:.2f}".format(RSS_train))

The RSS value for the training data set is 513.72


In [13]:
print ("The RSS value for the test data set is {0:.2f}".format(RSS_test))

The RSS value for the test data set is 154.97


In [14]:
for val in range(len(winedata.columns)-1):
    print (winedata.columns[val],':',"{0:.4f}".format(round(beta_hat[val],4)))

fixed acidity : -0.0023
volatile acidity : -1.1731
citric acid : -0.2621
residual sugar : 0.0282
chlorides : -2.1116
free sulfur dioxide : 0.0044
total sulfur dioxide : -0.0036
density : 4.8477
pH : -0.5639
sulphates : 0.8828
alcohol : 0.2891


## Questions

<span>__1. What are the qualitative results from your model? Which features seem to be most
important? Do you think that the magnitude of the features in X may affect the
results (for example, the average total sulfur dioxide across all wines is 46.47, but
the average chlorides is only 0.087).__

- _Going by the coefficients beta for each variable, it seems that the density is the most important feature as it has the highest absolute value for beta. However, the magnitudes of the features are very different. Density has an average value of 0.996 which is very small compared to features like alcohol with a mean value of 10. The magnitude of these features have an impact on the magnitude of the coefficients and hence the coefficients cannot be directly compared to understand feature importance. For example, keeping all else constant, if density was multiplied by 10 and used in the regression, its coefficient would have been 10 times smaller (the units make a difference). Hence, when the magnitudes of the features are very different, some preprocessing like scaling/standardizing maybe required before coefficient comparison._

<span>__2. How well does your model fit? You should be able to measure the goodness of fit,
RSS, on both the training data and the test data, but only report the results on the
test data. In Machine Learning we almost always only care about how well the
model fits on data that has not been used to fit the model, because we need to use
the model in the future, not the past. Therefore, we only report performance with
holdout data, or test data.__
- _The RSS on the test data is 154.97. The RSS is a function of number of data points in the dataset, so it can mainly be used for comparison across different methods_

### Function to get the RSS given the arguments of the minimize function and the data

In [15]:
def RSS_minimize(beta0, X_test, y_test, solver='BFGS'):
    res = minimize(fun=RSS, x0=beta0, method=solver, args=(X_train,y_train))
    beta_hat = res.x
    return RSS(beta_hat, X_test, y_test)

### Calculating and printing the RSS for different starting points

In [16]:
RSS_1 = RSS_minimize(beta0=[1,20,-3,24,-95,1116,-7000,88,91,190,-311], X_test=X_test, y_test=y_test)
RSS_2 = RSS_minimize(beta0=list(range(10000,10011)), X_test=X_test, y_test=y_test)
RSS_3 = RSS_minimize(beta0=list(range(-20,-31,-1)), X_test=X_test, y_test=y_test)
RSS_4 = RSS_minimize(beta0=list(range(-20000,-20011,-1)), X_test=X_test, y_test=y_test)

In [17]:
print('The values of RSS for different starting points are {0:.2f}, {0:.2f}, {0:.2f}, {0:.2f}'.format(RSS_1, RSS_2, RSS_3, RSS_4))

The values of RSS for different starting points are 154.97, 154.97, 154.97, 154.97


### Timing the runs

In [18]:
%%timeit -n 100
RSS_1 = RSS_minimize(beta0=[1,20,-3,24,-95,1116,-7000,88,91,190,-311], X_test=X_test, y_test=y_test)

100 loops, best of 3: 26.6 ms per loop


In [19]:
%%timeit -n 100
RSS_2 = RSS_minimize(beta0=list(range(10000,10011)), X_test=X_test, y_test=y_test)

100 loops, best of 3: 30.7 ms per loop


In [20]:
%%timeit -n 100
RSS_3 = RSS_minimize(beta0=list(range(-20,-31,-1)), X_test=X_test, y_test=y_test)

100 loops, best of 3: 18.1 ms per loop


In [21]:
%%timeit -n 100
RSS_4 = RSS_minimize(beta0=list(range(-20000,-20011,-1)), X_test=X_test, y_test=y_test)

100 loops, best of 3: 31.2 ms per loop


<span>__3. Does the end result or RSS change if you try different initial values of β? What
happens if you change the magnitude of the initial β?__
- _For the different initial values used above, the end result and the RSS does not change. However, the execution time changes. Based on observation of the limited trials, it looks like the execution is faster if our initial values are relatively closer to the optimized beta values._

### Trying out different solver methods for the optimization

In [22]:
beta0 = np.random.normal(0, 1, X_train.shape[1])

In [23]:
solver_methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP']

In [24]:
solver_RSS = OrderedDict()
for solver_method in solver_methods:
    solver_RSS[solver_method] = RSS_minimize(beta0=beta0, X_test = X_test, y_test = y_test, solver=solver_method)

solver_RSS_print = [str(x)+": "+str(round(solver_RSS[x],2)) for x in solver_RSS]

print ('RSS for different solver methods:')
for val in solver_RSS_print:
    print (val)

RSS for different solver methods:
Nelder-Mead: 936.88
Powell: 154.98
CG: 155.05
BFGS: 154.97
L-BFGS-B: 154.99
TNC: 222.66
COBYLA: 340.31
SLSQP: 154.97


<span>__4. Does the choice of solver method change the end result or RSS?__
- _Based on the test performed, the choice of solver method does change the end result as the RSS changes_

## Regularization

### Function to estimate RSS given coefficients beta, data containing independent vars X and dependent var y and regularization parameter lambda_reg

In [25]:
def RSS_regularized(beta, X, y, lambda_reg, reg_type = 'l2'):
    """ (array, array, array) -> float
    Returns the Sum of Squared errors calculated using the coefficients beta, input data array X and the corresponding actual value of y    
    """
    yhat = estimate_yhat(beta, X)    
      
    if reg_type == 'l2':
        return np.sum((y - yhat) ** 2) + (lambda_reg * np.sum(beta**2))
    elif reg_type == 'l1':
        return np.sum((y - yhat) ** 2) + (lambda_reg * np.sum(np.abs(beta)))

In [26]:
def RSS_minimize_reg(beta0, X_test, y_test, lambda_reg=0.01, reg_type='l2'):
    res = minimize(fun=RSS_regularized, x0=beta0, args=(X_train, y_train, lambda_reg, reg_type))
    beta_hat = res.x
    return RSS(beta_hat, X_test, y_test)

### L2 Regularization

In [27]:
l2_reg_RSS_train = OrderedDict()
for lambda_reg in [0, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50]:
    l2_reg_RSS_train[lambda_reg] = RSS_minimize_reg(beta0=beta0, X_test = X_train, y_test = y_train, lambda_reg=lambda_reg, reg_type='l2')

In [28]:
l2_reg_RSS_train_print = [str(x)+": "+str(round(l2_reg_RSS_train[x],3)) for x in l2_reg_RSS_train]

print ('RSS on training dataset for different lambda values:')
for val in l2_reg_RSS_train_print:
    print (val)

RSS on training dataset for different lambda values:
0: 513.717
0.01: 513.721
0.05: 513.797
0.1: 514.002
0.5: 517.052
1: 520.614
5: 531.618
10: 537.36
50: 562.325


In [29]:
l2_reg_RSS_test = OrderedDict()
for lambda_reg in [0, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50]:
    l2_reg_RSS_test[lambda_reg] = RSS_minimize_reg(beta0=beta0, X_test = X_test, y_test = y_test, lambda_reg=lambda_reg, reg_type='l2')

In [30]:
l2_reg_RSS_test_print = [str(x)+": "+str(round(l2_reg_RSS_test[x],3)) for x in l2_reg_RSS_test]

print ('RSS on training dataset for different lambda values:')
for val in l2_reg_RSS_test_print:
    print (val)

RSS on training dataset for different lambda values:
0: 154.972
0.01: 154.922
0.05: 154.752
0.1: 154.593
0.5: 154.282
1: 154.485
5: 155.452
10: 156.023
50: 160.531


<span>__Try adding in an L2 (aka Ridge) regularization penalty to your model above to create a new,
regularized model. See equation 3.41 for guidance. You will need to choose a value of
lambda, so start with something small, like 0.01. 
<br><br> 1. How does RSS on the training data change? How does RSS on the test data change?__

- _On adding an L2 penalty of 0.01, the training RSS increased marginally while the test RSS decreased marginally. _

<br> __2. What happens if you try different values of lambda? Can you roughly tune lambda to get
the best results on the test data?__

- _On testing different values of lambda and looking at performance on the test set (using RSS) , we can use a lambda value of 0.5 that seems to provide a relatively lower RSS(compared to other values of lambda). Also, as lambda increases, the test RSS first decreases and then continues to increase indicating that there may be an optimum point_


### L1 Regularization

In [38]:
l1_reg_RSS_train = OrderedDict()
for lambda_reg in [0, 0.01, 0.05, 0.1, 0.5, 1, 2.5, 5, 7.5, 10, 50]:
    l1_reg_RSS_train[lambda_reg] = RSS_minimize_reg(beta0=beta0, X_test = X_train, y_test = y_train, lambda_reg=lambda_reg, reg_type='l1')

In [39]:
l1_reg_RSS_train_print = [str(x)+": "+str(round(l1_reg_RSS_train[x],3)) for x in l1_reg_RSS_train]

print ('RSS on training dataset for different lambda values:')
for val in l1_reg_RSS_train_print:
    print (val)

RSS on training dataset for different lambda values:
0: 513.717
0.01: 513.717
0.05: 513.719
0.1: 513.725
0.5: 513.898
1: 514.441
2.5: 518.244
5: 523.764
7.5: 533.65
10: 541.195
50: 578.534


In [40]:
l1_reg_RSS_test = OrderedDict()
for lambda_reg in [0, 0.01, 0.05, 0.1, 0.5, 1, 2.5, 5, 7.5, 10, 50]:
    l1_reg_RSS_test[lambda_reg] = RSS_minimize_reg(beta0=beta0, X_test = X_test, y_test = y_test, lambda_reg=lambda_reg, reg_type='l1')

In [41]:
l1_reg_RSS_test_print = [str(x)+": "+str(round(l1_reg_RSS_test[x],3)) for x in l1_reg_RSS_test]

print ('RSS on training dataset for different lambda values:')
for val in l1_reg_RSS_test_print:
    print (val)

RSS on training dataset for different lambda values:
0: 154.972
0.01: 154.963
0.05: 154.928
0.1: 154.886
0.5: 154.591
1: 154.312
2.5: 154.089
5: 153.938
7.5: 155.089
10: 156.368
50: 163.498



__3. Now try using an L1 (aka Lasso) regularization penalty instead. See equation 3.51 for
example. Report your findings on how RSS changes, and if you can roughly tune lambda__

- _On testing different values of lambda and looking at performance on the test set (using RSS) , we can use a lambda value of 5 that seems to provide a relatively lower RSS (compared to other values of lambda). Also, as lambda increases, the test RSS first decreases and then continues to increase indicating that there may be an optimum point_


__4. Again, do you think that the magnitude of the features in X may affect the results with
regularization?__

- _By observation of the regularization objective, it seems that regularization focuses on shrinking the beta coefficient associated with each variable. But generally, the size of the beta coefficients is also a function of the magnitude of the features (with all else same, as units of one feature are changed, the betas get scaled accordingly). In other words, the size of the betas determine how cheap/expensive it is to include the feature as the betas contribute to the function we try to minimize. Hence, we need some form of scaling/standardization before we run regularized regression in order to treat the features fairly._
